# Spotify API Authentication

In [ ]:
!pip3 install requirements.txt

In [ ]:
import config

import requests
import pandas as pd

## Authorization

This dosent work for now and will be implemented in future versions.

### Workaround

* Signup for a Spotify developer account for your source account, and then create a new application from the dashboard.
* Copy the `client id` and `client secret` from the new app
* Head over to this [link](https://developer.spotify.com/documentation/web-api/reference/get-playlist) in your browser and open the developer console and head over to the network tab.
* Click on the `try it` option there, and get the `Bearer token` of the new request that would come from the developer window.
* Use this as the bearer token for the source account.

In [ ]:
client_id = config.client_id
client_secret = config.client_secret

## Get Users Playlists

In [ ]:
playlist_oauth_token = config.playlist_oauth_token

In [ ]:
# Get the list of playlists of the current user

url = "https://api.spotify.com/v1/me/playlists"
method = "GET"
header = {
    "Authorization" : f"Bearer {playlist_oauth_token}"
}

response = requests.get(url, headers=header)

In [ ]:
response.json()

In [ ]:
playlists_data = response.json()['items']

In [ ]:
df_columns = ["id", "Playlist Name", "Tracks Count"]
playlist_df = pd.DataFrame(columns=df_columns)
num_entries = 0

In [ ]:
for playlist in playlists_data :
    playlist_id = playlist['id']
    playlist_name = playlist['name']
    tracks_count = playlist['tracks']['total']
    
    playlist_df.loc[num_entries] = [playlist_id, playlist_name, tracks_count]
    num_entries += 1

In [ ]:
playlist_df

In [ ]:
playlist_df.to_csv('playlists.csv', index=False)

## Get the tracks from these playlists

In [ ]:
playlist_df = pd.read_csv('playlists.csv')

In [ ]:
playlist_df

In [ ]:
num_playlists = playlist_df.shape[0]

In [ ]:
df_columns = ["id", "Track Name", "Playlist Name"]
tracks_df = pd.DataFrame(columns=df_columns)
num_entries = 0

In [ ]:
# Get the tracks of the playlists of the current user

for i in range(num_playlists) :
    cur_playlist_id = playlist_df.iloc[i]['id']
    cur_playlist_name = playlist_df.iloc[i]['Playlist Name']
    cur_tracks_count = playlist_df.iloc[i]['Tracks Count']
    
    offset = 0
    while offset < cur_tracks_count :
        url = f"https://api.spotify.com/v1/playlists/{cur_playlist_id}/tracks"
        method = "GET"
        header = {
            "Authorization" : f"Bearer {playlist_oauth_token}"
        }
        query_params = {
            'offset' : str(offset)
        }

        response = requests.get(url, headers=header, params=query_params)
        if not (200 <= response.status_code < 300) :
            print("Error")
    
        tracks_data = response.json()['items']

        print(f"Recieved {len(tracks_data)} tracks from the playlist : {cur_playlist_name}")

        for track in tracks_data :
            cur_track_id = track['track']['id']
            cur_track_name = track['track']['name']

            tracks_df.loc[num_entries] = [cur_track_id, cur_track_name, cur_playlist_name]
            num_entries += 1
        
        offset += 100

In [ ]:
tracks_df.loc[tracks_df['Playlist Name'] == 'TS']

In [ ]:
tracks_df.to_csv('tracks.csv', index=False)

## Create Playlists and Populate them

We create new playlists in the new spotify account and then transfer this content into the new account using by creating the playlists accordingly and then populating it with the tracks accordingly

### Authorization Workaround

* Note that we do not have to create a new application, but do sign up for a new developer profile in the new account.
* Run a request from this [page](https://developer.spotify.com/documentation/web-api/reference/get-current-users-profile) and get the `user id` for the new account from the results of this request and the `bearer token` from the developer window.

In [ ]:
# Get the Authentication token for the second account

account_2_playlist_oauth_token = config.account_2_playlist_oauth_token
account_2_user_id = config.account_2_user_id

In [ ]:
tracks_df = pd.read_csv('tracks.csv')

In [ ]:
# Get the unique names of the playlists

playlists = tracks_df['Playlist Name'].unique()

In [ ]:
for playlist_name in playlists :
    
    # Create a new playlist
    url = f"https://api.spotify.com/v1/users/{account_2_user_id}/playlists"
    method = "POST"
    header = {
        "Authorization" : f"Bearer {account_2_playlist_oauth_token}",
        "Content-Type" : "application/json"
    }
    body = {
        "name" : playlist_name,
        "public" : "false",
        "collaborative" : "false",
    }
    response = requests.post(url, headers=header, json=body)
    if 200 <= response.status_code < 300 :
        print(f"Successfully created : {playlist_name}")
    playlist_id = response.json()['id']
    
    # Get the id's of the songs of the current playlist
    cur_tracks = tracks_df.loc[tracks_df['Playlist Name'] == playlist_name]
    body = {'uris' : []}
    
    # Spotify API Allows us to pass a maximum of 100 tracks at once
    cur_tracks_id_list = [cur_tracks.iloc[i]['id'] for i in range(cur_tracks.shape[0])]
    offset = 0
    
    # Add these tracks in batches of 100
    while offset < len(cur_tracks_id_list) :
        body['uris'] = [f"spotify:track:{cur_track_id}" for cur_track_id in cur_tracks_id_list[offset:min(offset+100, len(cur_tracks_id_list))]]
        url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks"
        method = "POST"
        header = {
            "Authorization" : f"Bearer {account_2_playlist_oauth_token}",
            "Content-Type" : "application/json"
        }
        response = requests.post(url, headers=header, json=body)
        
        cur_num_tracks = min(offset+100, len(cur_tracks_id_list)) - offset
    
        if 200 <= response.status_code < 300 :
            print(f"Successfully added {cur_num_tracks} tracks to {playlist_name}")
        
        # Increment offset to add the next batch
        offset += 100